# Algoritmos de colmena - Juan Antonio Silva Luján y Luis Bote Gallardo



---





![Fig01.Imagen Presentación](http://2.bp.blogspot.com/-SnUkuHoTcSQ/Uj0L1q8wGKI/AAAAAAAAANY/Rc5VQe5l8aM/s1600/hormigas.jpg)

## 1. Introducción al problema


El análisis del algoritmo que nos ocupa se basa en la observación del comportamiento de las hormigas.
Inicialmente, esta colmena se mueve de forma aleatoria, no obstante, cuando las hormigas comienzan a moverse por caminos prometedores, comienzan a depositar feromonas que otras hormigas asumirán para poder "localizar" los caminos más prometedores.
Así mismo, las feromonas, se van evaporando, perdiendo su valor a largo plazo.

## Evaporación de las feromonas

Mediante la función evaporaferomonas podemos introducir la feromona concreta que deseamos evaporar aplicando un coeficiente del 10% de disminución.
La feromona actual se encuentra en uno nodo. Esta técnica es aplicada para evitar la aparición de óptimos locales.




---



In [0]:
#Realizamos la función de ir evaporando las feomonas, se van multiplicando
#por una constante con valor 0.9 = Coeficiente 10%
def evaporaferomonas(feromonas):

  coeficienteReduccion=0.9
  for lista in feromonas:           #Recorremos la lista de feromonas, en cada iteración
    for i in range(len(lista)):     #vamos aplicando una reducción del 10% del valor
      lista[i] = lista[i] * coeficienteReduccion

## Rastrear Feromonas depositadas

Durante el proceso de ejecución del algoritmo, cada hormiga es capaz de ir rastreando las feromonas depositadas en las aristas por otras hormigas.
Este método ayudará a actualizar estas feromonas por cada hormiga ejecutada.

In [0]:
#Actualización de la estructura matricial de feromonas
#dado un camino croncreto
def rastroferomonas(feromonas, camino, dosis):

  for i in range (len(camino) - 1):
    feromonaR = feromonas[camino[i]][camino[i+1]]
    feromonaR = feromonaR + dosis



---



## Hormigas

Implementación del algoritmo del viajante aplicado a la colonia de hormigas. Se recibe por parámetros un array bidimensional

In [0]:
#El método retornará una estructura (tupla) con el camino
#evaluado más prometedor de todos los obtenidos así como 
#la distancia total que tiene.

def hormigas(matriz, iteraciones, distMedia):

  #Dclaración/Inicialización de matriz para las feromonas
  #por defecto, se inicializará la matriz vacía
  n = len(matriz)
  feromonas = [[0 for i in range(n)] for j in range(n)]

  #Declaración del camino más prometedor y su longitud, 
  #inicialmente, se le asignará el máximo valor posible
  #y se irá adaptando a lo largo de la ejecución cuando se
  #vayan actualizando los correspondientes pesos.
  camino_prometedor = []
  distancia_mejor = sys.maxsize

  #Recorremos todas las iteraciones generando nuevas hormigas que 
  #irán recorriendo un camino, en caso de tratarse de un camino 
  #prometedor que supere al anterior, deposita una feromona que
  #tendrá "mejor" en los casos de que los caminos sean más cortos.
  for iter in range(iteraciones):
    (camino,distancia) = eligecamino(matriz, feromonas)
    if (distancia <= distancia_mejor):
      camino_prometedor = camino
      distancia_mejor = distancia
    
    dis = distMedia/distancia
    rastroferomonas(feromonas, camino, dis)

    # En cualquier caso, las feromonas se van evaporando
    evaporaferomonas(feromonas)

  # Se devuelve el mejor camino que se haya encontrado
  return (camino_prometedor, distancia)



---



## Elección del Nodo

La función de elección del nodo sirve para decidir hacia que nodo se expandirá la hormiga, el método retornará una estructura (lista) con todas las opciones posibles expandibles teniendo en cuenta los valores:


1.   Ciudades
2.   Feromonas



In [0]:
import random, sys, math
#Expande un nodo concreto (camino) analizando los pesos de las feromonas y 
#los nodos que ya han sido visitados anteriormente
def eligenodo(valores, feromonas, visitados):

  #Declaración de valor de cada ciudad
  LValores = []
  LAccesibles = []
  actual = visitados[-1]

  # Declaración de valores de cada ciudadn(alfa -> feromonas & beta-> valor)
  ALPHA = 1.0
  BETA = 2.0

  #Se recorre la estructura, en cada iteración, se calcula 1.0 + la feromona 
  #correspondiente elevado al valor ALPHA, a continuación, se calcula el valor
  #del peso mediante la división de 1.0 entre el valor que corresponde elevado
  #a BETA
  for i in range(len(valores)):
    if i not in visitados:
      ferom = math.pow((1.0 + feromonas[actual][i]), ALPHA)
      ponderacion = math.pow(1.0/valores[actual][i], BETA)*ferom
      LAccesibles.append(i)
      LValores.append(ponderacion)

  #Introducimos en valor una de las opciones de forma aleatoria pero teniendo en
  # consideración el valor del peso.
  valor = random.random() * sum(LValores)
  acumulador = 0.0
  i = -1
  while valor > acumulador:
    i += 1
    acumulador += LValores[i]
  return LAccesibles[i]



---



## Elección del camino
Este método actúa a partir de la ejecución de la función anterior (eligenodo), usa un nodo retornado por la estructura para decidir cuales son las ciudades más prometedoras para expandir.
El método retorna cual será el camino elegido así como su distancia tota. 

In [0]:
#Recibe una matriz y las feromonas para 
def eligecamino(matriz, feromonas):

  #Declaración/Inicialización del camino prometedor y su distancia,
  #por defecto, inicializamos a 0, este valor irá incrementándose
  #a lo largo de la ejecución del algoritmo.
  camino = [0]
  distancia_acum = 0.0

  #Mientras que la estructura que almacena los caminos sea menor que
  #la matriz, en cada iteración se invoca a la función eligenodo
  #para ir acumulando la distancia según la iteración actual.
  while (len(camino) < len(matriz)):
    nodo = eligenodo(matriz, feromonas, camino)
    distancia_acum = distancia_acum + matriz[camino[-1]][nodo]
    camino.append(nodo)

  #Cuando terminamos con todas las iteraciones anteriores, se vuelve
  #a ubicar por defecto el nodo 0
  distancia_acum = distancia_acum + matriz [camino[-1]][0]
  camino.append(0)
  return (camino, distancia_acum)



---



---



## Matriz distancias (Opción 1)


Sirve para no tener que introducir todos los datos relativos a las ciudades manualmente, con este función, se genera una matriz con los valores aleatoriamente para realizar la ejecución de la simulación del programa.


In [0]:
#A la hora de utilizar las ciudades y generar la matriz, tenemos dos opciones:
#La primera

#Se crea una matriz de N x N donde N es el número de ciudades
#asignando valores de forma aleatoria 
from random import randrange
def matrizdistancias(N, distanciaMaxima):

  matriz = [[0 for i in range(N)] for j in range (N)]
  for i in range(N):
    for j in range(i):
      matriz[i][j] = 1+randrange(100)
      matriz[j][i] = matriz[i][j]

  #Impresión de la matriz resultante
  print("Matriz de ciudades: ")
  print()
  for i in matriz:
    print(i)  
 
  return matriz

## Matriz distancias (Opción 2)


Carga la matriz distancias desde un archivo txt guardado en el entorno de ejecución

In [0]:
def matrizdistancias2():
  # Mostramos por pantalla lo que leemos desde el fichero
  print('>>> Cargamos las matriz del txt')
  filename ='/content/ciudadesEjemplo.txt'

  f = open('/content/ciudadesEjemplo.txt')
  # Primera lectura para obviar la cabecera
  ciudades = f.readline()
  print(ciudades)

  # Cargamos los datos
  data = f.read().strip()

  # Se cierra el fichero
  f.close()

  #Separamos los datos en un array
  matriz = [[int(num) for num in line.strip().split(',')] for line in data.split('\n')]
  for i in matriz:
    print(i)  
  return (matriz,ciudades) 



In [0]:
# Busca el valor máximo de una matriz
def maxMatriz(matriz):
  maximo = matriz[0][0]
  for i in range(len(matriz)):
    for j in range(len(matriz[0])):
      if matriz[i][j] > maximo:
          maximo = matriz[i][j]
  print()
  print('Valor máximo de la matriz', maximo)
  return maximo


#Ejecución del algoritmo

Ejecutamos el método principal que llamará a los métodos anteriores, aquí podemos decidir que opción utilizar

In [0]:
from time import time

# Start counting.
start_time = time()

#Declaración de variables y asignación de valores de ejemplo

iteraciones  = 2000
hormigasMAX = 5


#Opción 1
numCiudades  = 12
MAXdistancia = 300
ciudades = matrizdistancias(numCiudades, MAXdistancia)





#Opción 2
#ciudades,numCiudades=matrizdistancias2()
#numCiudades = len(numCiudades)
#MAXdistancia = maxMatriz(ciudades)


#Cálculo de la distancia media -> Cantidad de ciudades multiplicado
#por el valor de la distancia máximo, todo ello partido por 2.
numHormigas=hormigasMAX
if hormigasMAX < numCiudades:
  numHormigas = numCiudades

distMedia = numCiudades*MAXdistancia/2
for i in range(1,numHormigas):
  (camino, longCamino) = hormigas(ciudades, iteraciones, distMedia)
  i = i+1

#Impresión por pantalla de los valores retornados tras la ejecución
#del programa
print()
print("____________________________________________________")
print()
print("Camino -> ", camino)
print()
print("Longitud del camino:    ", longCamino)
print()

# Calculate the elapsed time.
elapsed_time = time() - start_time
print()
print("Tiempo de ejecución: %0.5f segundos." % elapsed_time)

Matriz de ciudades: 

[0, 30, 63, 69, 54, 56, 67, 21, 18, 56, 40, 69]
[30, 0, 43, 89, 86, 16, 55, 50, 25, 91, 8, 57]
[63, 43, 0, 19, 1, 70, 29, 68, 23, 90, 52, 88]
[69, 89, 19, 0, 55, 96, 21, 31, 94, 38, 99, 11]
[54, 86, 1, 55, 0, 66, 46, 38, 8, 77, 10, 28]
[56, 16, 70, 96, 66, 0, 49, 45, 6, 59, 70, 23]
[67, 55, 29, 21, 46, 49, 0, 80, 83, 63, 81, 67]
[21, 50, 68, 31, 38, 45, 80, 0, 8, 34, 1, 58]
[18, 25, 23, 94, 8, 6, 83, 8, 0, 39, 18, 85]
[56, 91, 90, 38, 77, 59, 63, 34, 39, 0, 2, 83]
[40, 8, 52, 99, 10, 70, 81, 1, 18, 2, 0, 41]
[69, 57, 88, 11, 28, 23, 67, 58, 85, 83, 41, 0]

____________________________________________________

Camino ->  [0, 8, 4, 2, 6, 3, 11, 5, 1, 10, 9, 7, 0]

Longitud del camino:     302.0


Tiempo de ejecución: 1.69543 segundos.


#fichero de prueba

copiar y pegar el texto de debajo en un fichero "ciudadesEjemplo.txt" y cargarlo desde el modulo de la opcion 2

ciudades = ["Badajoz","Caceres","Merida","Plasencia","Malpartida-de-Caceres","Gevora"]
0,93,66,170,94,7
93,0,75,80,11,85
66,75,0,150,74,65
170,80,150,0,87,163
94,11,74,87,0,88
7,85,65,163,88,0 
